# Paraphrase Italian Dataset

In [2]:
import pandas as pd

In [67]:
# Use 'skip' to skip problematic lines and continue parsing
df = pd.read_csv('PACCSS-IT.txt', sep='\t', usecols=range(7), quoting=3)
df.head()

,Sentence_1,Sentence_2,Cosine_Similarity,Confidence,Readability_1,Readability_2,(Readability_1-Readability_2)
0,Ma questo a cosa servirebbe ?,A che servono queste cose ?,0.833333,0.803743,0.994767,0.013161,0.981605
1,"Salve, avrei bisogno di una informazione piutt...",Ho bisogno di una informazione urgente .,0.836660,0.811129,0.982944,0.012627,0.970317
2,Ciao a tutti avrei bisogno di un consiglio .,Ho bisogno di un suo consiglio .,0.755929,0.811662,0.984332,0.014420,0.969912
3,Possibilmente uno che avesse bisogno dell' aiu...,Ho bisogno di un vostro aiuto .,0.801784,0.826830,0.982555,0.014647,0.967908
4,Questa sarebbe una cosa positiva.,Questa era una nuova cosa .,0.771517,0.800786,0.997256,0.030448,0.966808


In [68]:
min_confidence = df['Confidence'].min()
max_confidence = df['Confidence'].max()
min_cosine = df['Cosine_Similarity'].min()
max_cosine = df['Cosine_Similarity'].max()

# Display the results
print(f"Confidence range: {min_confidence} to {max_confidence}")
print(f"Cosine Similarity range: {min_cosine} to {max_cosine}")

Confidence range: 0.800002575466 to 0.9999999
Cosine Similarity range: 0.654653670708 to 0.949288905069


## Positive Class Creation

In [69]:
# drop the last three columns
df = df.iloc[:, :-3]
df.head()

,Sentence_1,Sentence_2,Cosine_Similarity,Confidence
0,Ma questo a cosa servirebbe ?,A che servono queste cose ?,0.833333,0.803743
1,"Salve, avrei bisogno di una informazione piutt...",Ho bisogno di una informazione urgente .,0.836660,0.811129
2,Ciao a tutti avrei bisogno di un consiglio .,Ho bisogno di un suo consiglio .,0.755929,0.811662
3,Possibilmente uno che avesse bisogno dell' aiu...,Ho bisogno di un vostro aiuto .,0.801784,0.826830
4,Questa sarebbe una cosa positiva.,Questa era una nuova cosa .,0.771517,0.800786


In [70]:
confidence_treshold = 0.90
cosine_threshold = 0.80

In [71]:
# Filter the DataFrame based on the thresholds
filtered_df = df[(df['Confidence'] >= confidence_treshold) & (df['Cosine_Similarity'] >= cosine_threshold)]

len(filtered_df)

14545

In [72]:
# pick 100 random samples
sample = filtered_df.sample(n=100, random_state=1)

#show the sample
sample

,Sentence_1,Sentence_2,Cosine_Similarity,Confidence
35173,Ma questo è solo il primo passo .,Ed è solo il primo passo .,0.801784,0.971201
42931,"Posto ai voti , l' emendamento 6.1 , viene res...",Posto ai voti è respinto l' emendamento 1.1 .,0.843274,0.900495
24230,"La Commissione prosegue l' esame , rinviato , ...",La Commissione prosegue l' esame rinviato nell...,0.886405,0.927586
42879,Questo è un punto di estrema importanza .,è un punto di grande importanza .,0.801784,0.964339
59391,"E allora , un "" altro mondo "" sarà possibile .","E allora , un altro mondo sarà possibile .",0.948683,0.931301
...,...,...,...,...
45885,"Si tratta , dunque , di un passaggio decisivo .",Si tratta di un passaggio decisivo .,0.881917,0.951611
15271,"Il fine , però , non giustifica i mezzi .",Il fine non giustifica mai i mezzi .,0.801784,0.950393
12011,Le cose però non starebbero così .,Le cose non stavano esattamente così .,0.857143,0.938938
30628,Questa però è opinione mia .,Questa è una mia opinione .,0.833333,0.957264


In [73]:
# add a column label with value 1
filtered_df['label'] = 1

<ipython-input-73-638f61ff7a66>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['label'] = 1


## Negative Class Creation

### Supervied Random Sampling

We take a sentence A from the corpus and randomly sample a sentence B to associate with it. We exclude sentences with a cosine similarity greater than a certain threshold to avoid false negatives. Of course, the sentence pair should not be already present in the dataset.

In [74]:


def generate_random_sentence_pair():
    # pick a random Sentence_1
    s1 = df.sample(n=1)['Sentence_1'].values[0]
    # pick a random Sentence_2
    s2 = df.sample(n=1)['Sentence_2'].values[0]
    return s1, s2


In [75]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

threshold = 0.70

negative_df = pd.DataFrame(columns=['Sentence_1', 'Sentence_2', 'Cosine_Similarity', 'label'])

while (len(negative_df) <= len(filtered_df)): # ratio 1:1 with the positive class
    s1, s2 = generate_random_sentence_pair()
    # check if the pair is not already in the corpus
    if not ((df['Sentence_1'] == s1) & (df['Sentence_2'] == s2)).any():
        # check if the pair is not already generated
        if not ((negative_df['Sentence_1'] == s1) & (negative_df['Sentence_2'] == s2)).any():
            # calculate the cosine similarity between s1 and s2
            emb_s1 = model.encode(s1)
            emb_s2 = model.encode(s2)
            cosine_similarity = model.similarity(emb_s1, emb_s2)
            # check if the cosine similarity is less than the threshold
            if cosine_similarity < threshold:
                row = pd.DataFrame([{'Sentence_1': s1, 'Sentence_2': s2, 'Cosine_Similarity': cosine_similarity, 'label': 0}])
                negative_df = pd.concat([negative_df, row], ignore_index=True)

negative_df.head()

,Sentence_1,Sentence_2,Cosine_Similarity,label
0,Vi ringrazio per l' attenzione e saluto cordia...,Della riunione viene redatto apposito verbale .,[[tensor(0.5547)]],0
1,La partita è apertissima » .,Da dove vengono questi soldi ?,[[tensor(0.3595)]],0
2,"Allora , qual è il problema ?",L' articolo 21 è sostituito dal seguente : - «...,[[tensor(0.4530)]],0
3,"Ringraziando- la per l' attenzione , La saluto...",Quali sono le conseguenze pratiche ?,[[tensor(0.5503)]],0
4,Ma questa è una idea errata » .,Ma quali saranno le conseguenze ?,[[tensor(0.4357)]],0


In [76]:
# save the positive and negative samples to a csv file
filtered_df.to_csv('PACCSS-IT-positive.csv', index=False)
negative_df.to_csv('PACCSS-IT-negative.csv', index=False)

In [ ]:
# read data from the csv files
positive_df = pd.read_csv('PACCSS-IT-positive.csv') 
negative_df = pd.read_csv('PACCSS-IT-negative.csv')

# merge the positive and negative samples
final_df = pd.concat([positive_df, negative_df], ignore_index=True)

# drop the 'Cosine_Similarity', 'Confidence' column
final_df = final_df.drop(columns=['Cosine_Similarity', 'Confidence'])

# shuffle the final DataFrame
final_df = final_df.sample(frac=1, random_state=42).reset_index(drop=True)

# save the final DataFrame to a csv file
final_df.to_csv('PACCSS-IT-final.csv', index=False)



,Sentence_1,Sentence_2,label
0,Ma questo non fu un caso isolato .,I fatti mi sembrano questi .,0
1,"L' emendamento 10.1 , posto ai voti , è approv...",Posto ai voti è approvato l' emendamento 01.1 .,1
2,Il giudizio complessivo è : ottimo .,Il giudizio complessivo è quindi ottimo .,1
3,Esaminati gli atti e documenti tutti della cau...,Esaminati gli atti ed i documenti di causa .,1
4,Ma questa è solo la mia opinione .,Ma è solo una mia opinione .,1
